# Machine learning classification

The spam folders on email systems were really bad in the recent past. Emails were filtered by a rule, like looking for key words (e.g., soliciting money) or by manually identifying spam emails that were being spread widely through email. The classification of our emails, by machines, into relevant emails and spam, has advanced to such a degree that we now take for granted that a lot of the spam email we receive is automatically routed to the spam folder, needing little oversight from us. These machine classifiers use algorithms that are applicable to a wide variety of fields: written language; spoken language (e.g. "Hello Google", "Alexa", "Siri"); navigating driverless cars. Today, we'll also use them to understand brain activity. 

Classification of fMRI data has led to a revolution in the past two decades, with thousands of studies using classifiers to test if conditions evoke the different patterns of activity in the brain. Here we will be applying classifiers to movie-data, which is not a typical application. By doing so we will gain an acute familiarity with some of the assumptions of classifiers and how to deal with them.

Throughout today you will see how *powerful* machine learning tools can be, but we hope you realize that *they are not magic*. In many ways, the actual operation being used is simple and familar: the classifier we will be using is a fancy logistic regression. However, when used correctly, machine learning can make unique and incisive contributions.

We will be using the Sherlock dataset today. Review details about it from week 2 if you need a reminder.

## Goal of this script
Using this script you will learn to use a classifier on a dataset. Specifically, we will accomplish the following:  
>1. Load and prepare the data for classification
>2. Learn about the importance of independence and how to avoid double dippling
>3. Apply a classifier within participant


## Table of Contents
[1. Loading metadata](#load_data)  

[2. Independent observations](#independence)  

[3. Loading and masking functional data](#mask)

[4. Classification](#classification)  

[5. Double dipping](#dipping)

[6. Classification in all participants](#between)

Exercises
>[Exercise 1](#ex1)  [2](#ex2)  [3](#ex3)   [4](#ex4)  [5](#ex5)   [6](#ex6)   [7](#ex7) [8](#ex8) [9](#ex9)

[Novel contribution](#novel)




In [ ]:
# We need to add the utils to the path
import sys
sys.path.insert(0, '..')

# Import utils
from utils import * 

# Preset the some information about the experiment and analysis. These won't necessarily make sense now, but we will refer to them later
TR_duration = 1.5 # How many seconds is each TR
lag_shift = 6 # How many seconds is the shift you will apply to the labels to account for hemodynamic lag?
first_segment_duration = 946 # How long was the first movie segment
ppt_num = 17 # How many participants are there?

# Preset for later
TR_condition = {} 
observation_TRs_raw = {}
observation_TRs_shifted = {}
func_mat = {}
labels = {}

%matplotlib inline 
%autosave 5

## 1. Loading metadata<a id="load_data"></a>

We are going to load in information about the Sherlock dataset (AKA metadata). In particular, we are going to load in a spreadsheet that contains information about what was on the movie at each individual time point. This was created manually with excruciating effort 

The [pandas](https://pandas.pydata.org/) package is an *extremely* useful tool for interacting with spreadsheet data like CSVs (comma separated values format) or excel. It is also useful as a way to store data in tables. Below we load in the data using pandas

In [ ]:
# Load in the CSV file
pd.set_option('display.max_rows', 1000)
df = pd.read_csv('%s/derivatives/event_file.csv' % sherlock_dir)
df

Look closely at the table printed out above. Each row is a segment of the movie in which something is happening (defined in the 'Scene details - A level' column). We can explore the data in lots of different ways. For instance below, we can count the number of segments in which the scene was indoors vs outdoors

In [ ]:
df['Space_Indoor_Outdoor'].str.split(expand=True).stack().value_counts()

<div class="alert alert-block alert-warning">
    <strong>What just happened?</strong> In the above line of code, we have a very complicated statement that produces the desired output. One of the great things about Python is that it lets you pipe outputs into arguments directly. For instance, <code>.str</code> is a function that is applied to the pandas directory. The output of that function is then directly passed to <code>.split(expand=True)</code>, and so on. The result is that you can create short code, but at the cost of clarity. It is up to you how you decide to code but in these notebooks we will rarely squish functions together like this because it isn't didactic </div>

**Exercise 1**<a id="ex1"></a>: Below we ask you a few questions about the spreadsheet to make sure you can navigate it
    
**Q:** What do you think "Start Time (s)" and "End Time (s)" mean?  
**A:**  

**Q:** What TR of the movie does the 628th segment start on (use the `Segment Number` column to find this segment)?  
**A:**  

**Q:** What is happening during the 166th segment? Specifically you should report the "Scene details - B level" content and check the `Segment Number` column to get the number. To report your answer, index the pandas variable (`df`) using pandas indexing. This page is overkill but has a lot of the information you need: https://pandas.pydata.org/docs/user_guide/indexing.html  

In [ ]:
# Insert code to report the answer

For classification, we want to find two labels (i.e., stimulus categories) that we think will make good candidates for decoding. There are lots of options (e.g., Indoor vs outdoor, Sherlock vs Watson, Close up vs Long camera angle), but we are going to *make* a different category: Is someone speaking vs is no one speaking. Our metadata right now isn't set up for this: The column `Name_Speaking` lists which speaker is talking and is then blank when no one is talking. 

Below, we are going to make a new column in our dataset that simplifes `Name_Speaking`. We will make `is_speaking` a binary variable that is `1` when anyone is speaking and `0` when no one is speaking. 

In [ ]:
# Preset a list
binarized_label = []

# Loop through all the segments
for speaker in df['Name_Speaking']:
    
    # When no one is speaking, the label is 'nan', nans are weird python values where nan != nan
    if speaker != speaker:
        binarized_label += [0]
    else:
        binarized_label += [1]

# Add the new column to the dictionary and visualize it again
df['is_speaking'] = binarized_label
df

**Self-study:** In the above code we used `+=`. In Python, `x = x + 1` is the same as `x += 1`. In fact, it is possible to use `/=`, `*=`, `-=` and `+=`. In this case, we are appending a new value to a list by doing this, just like if we did: `binarized_label = binarized_label + [1]`

With this newly created column, we want to make a vector where each element of the vector corresponds to a TR in our fMRI data and tells us whether someone was or wasn't speaking. The current column doesn't do that because each row refers to a segment which can be any duration, not a single TR.

In the code below we make the vector we need by finding the start and end TR of each segment, and then apply the condition labels to each TR in the segment. The result is the `TR_condition` variable

In [ ]:
# What is the condition you are testing
condition_key = 'is_speaking'

# Preset the variable where each TR is a nan
TR_condition[condition_key] = np.ones((int(np.nanmax(df[' End Time (s) ']) / TR_duration, ))) * np.nan # Make an array of nans that is the length of the movie

# Cycle through each condition
for segment_counter, condition in enumerate(df[condition_key]):
    
    start_time = df['Start Time (s) '][segment_counter]
    end_time = df[' End Time (s) '][segment_counter]
    num_TR = np.round((end_time - start_time) / TR_duration)

    start_idx = int(np.round(start_time / TR_duration))
    end_idx = int(np.round(end_time / TR_duration))

    # Store the condition labels
    TR_condition[condition_key][start_idx:end_idx] = condition

**Self-study:** In the above function we used `enumerate`. This is an exceptually useful function for for loops. Read up on it [here](https://www.geeksforgeeks.org/enumerate-in-python/)

**Exercise 2**<a id="ex2"></a>: Use the `TR_condition` variable that was just made to create a plot showing the TR labels, with TR on the x axis and the condition on the y axis. This plot should clearly show how labels alternate during the experiment, and thus you should fuss around with how it looks until it is clear. Here are some specific things you need to do: 

>Use `plt.yticks` to specify the condition labels.  
>Resize the figure using `figsize`    
>Add axis labels.  

In [ ]:
# Insert code here

**Exercise 3:<a id="ex3"></a>** Rerun the steps above to create a new column called `is_music` which takes the `Music_Presence` variable and makes it binary. Store it in `TR_condition['is_music']`. Also make the same kind of plot like you did above.

In [ ]:
# Insert code here

## 2. Independent observations<a id="independence"></a>

As with many statistical methods, machine learning analyses require that our observations are independent. Two processes are independent in a statistical sense when the variation in one process does not affect the variation of another process. One way to assess independence is to consider whether the variation in the two processes are uncorrelated, and by the same token, processes are non-independent if they are correlated. 

Many analyses with human participants breach the assumption of independence. For instance, whenever we take multiple measurements from a participant (e.g., response times to trials) then they are non-independent since they come from the same participant. For this reason, some within participant analyses (e.g., correlating reaction times on trials with the trial difficulty) would violate assumptions of independence.  This is *bad* in an objective sense, but many analyses are robust to minor violations of independence. 

Adjacent time points in fMRI data are not independent because of autocorrelation, whereas far apart time points are independent (or at least, are *more* independent). Autocorrelation means that the events that happen at timepoint *t* influence the events that happen at timepoint *t+1*. Below we show you an example that gives you an intuition for why this is.

In [ ]:
# First, lets make a function that will help us with this demonstration
def make_HRF_response(event_times, event_magnitudes, duration=40):
    # Take in the event times and output a hemodynamic response from these events
    #
    # event_times is a list of onsets of events (in seconds)
    # event_magnitudes is a list of the expected response magnitude of events
    # duration is the duration of time being simulation
    
    # Specify when events occur and what their magnitude is
    stim_function = np.zeros((duration, 1))
    stim_function[event_times, 0] = event_magnitudes
    
    # Convolve that function with the HRF
    response = sim.convolve_hrf(stim_function, 
                                tr_duration = 1.5, 
                                temporal_resolution=1,
                               )
    
    # Scale it back to what it should be
    response *= np.max(event_magnitudes)
    
    # Return the convolution
    return response


In [ ]:
# Imagine two events occurring in the movie that evoke different magnitudes of response in a part of the brain
event_A_onset = 0 # When does event A onset
event_B_onset = 8 # When does event B onset
event_A_magnitude = 1 # How strongly does this brain region respond to event A
event_B_magnitude = 0.5 # How strongly does this brain region respond to event B

# Now plot the data with events A and B as if they occur in isolation, and then when they occur back to back.

plt.figure()

# Plot the individual responses using the output of the function directly in the plot
plt.plot(make_HRF_response([event_A_onset], [event_A_magnitude]), color='r')
plt.plot(make_HRF_response([event_B_onset], [event_B_magnitude]), color='b')
plt.plot(make_HRF_response([event_A_onset, event_B_onset], [event_A_magnitude, event_B_magnitude]), color='m')

plt.legend(['Event A only', 'Event B only', 'Event A and B'])

Look at the peaks of the magenta line, occuring at ~6s and ~14s. This corresponds to the peak response to the events A and B, including the hemodynamic lag (remember a discussion on this from Week 1). Based on this plot above, you can see that when event B occurs on its own (blue), it gets a stronger response at peak than when it is presented after event A (magenta).

**Exercise 4:<a id="ex4"></a>** Play around with the onsets of event A and B to find the answers to the following questions within the simulation (your answers should be integers).

**Q:** How many seconds must A and B be apart for the responses at peak to be independent (i.e., B is unaffected by A)?  
**A:**

**Q:** What is the maximum separation between the onsets of A and B such that the peak of A is unaffected by B, but with any shorter separation A is affected by B?  
**A:**  

Hopefully in responding to the above questions you will appreciate how important even a meagre amount of independence requires a substantial amount of time separation between observations.

In this vein, we will now subsample our `TR_condition` variable that we made before in such a way that we identify candidate TRs that are sufficiently separated in time. Specifically, we will identify time points that are at least 12s apart (8 TRs) and are surrounded by the same label for at least 3 TRs on both sides. The idea is that we will identify observations that are minimally effected by other observations, hence independent, but of course, it isn't *completely* independent.

We are going to add an additional wrinkle of complexity. You will remember that the movie was shown to participants in two segments. We are going to get the time points separately for the two segments. We will explain why in the classification section.

In [ ]:
# Set parameters up
TR_separation = 8 # What is the minimum number of TRs that separate two observations
TR_buffer = 3 # How many TRs on each side of the current TR must be the same condition for it to be used

condition_key = 'is_speaking'
observation_TRs_raw[condition_key] = {'first_segment': [], 'second_segment': []} # preset

# Cycle through each TR and evaluate whether it meets the two criteria we have established
for TR_counter in range(len(TR_condition[condition_key])):
    
    # Make sure there is space around the events to be considered at all (we include the lag_shift variable, which we will return to in a minute)
    if (TR_counter >= TR_buffer) and (TR_counter < len(TR_condition[condition_key]) - TR_buffer - 1):
        
        # What are the indexes for the buffer
        start_idx = TR_counter - TR_buffer
        end_idx = TR_counter + TR_buffer
        
        # Decide what segment the participant is in
        if (start_idx <= first_segment_duration) and (end_idx <= first_segment_duration): 
            # If the start and end index are BEFORE the end of the first segment, it is first segment
            segment_name = 'first_segment'
            
        elif (start_idx >= first_segment_duration) and (end_idx >= first_segment_duration): 
            # If the start and end index are AFTER the end of the first segment, it is second segment
            segment_name = 'second_segment'
            
        else:
            # If the buffer straddles between the first and second half, ignore it
            continue
            
        # CRITERIA 1: Check that all of the TRs are the same inside the buffer
        if len(np.unique(TR_condition[condition_key][start_idx:end_idx])) == 1:
            
            # CRITERIA 2: Now check that the last observation recorded wasn't closer than the minimum separation
            if np.all((TR_counter - TR_separation) >= np.asarray(observation_TRs_raw[condition_key][segment_name])):
                
                # Since it has met both criteria, add it to the list
                observation_TRs_raw[condition_key][segment_name] += [TR_counter]
    

Now that we have the TRs, let's count how many of each condition we have. 

In [ ]:
# Combine the labels across segments
observation_labels = []
for segment_name in observation_TRs_raw[condition_key].keys():
    observation_labels += list(np.asarray(TR_condition[condition_key])[np.asarray(observation_TRs_raw[condition_key][segment_name])])
    
# Plot the number of labels
plt.hist(observation_labels)
plt.title('Number of labeled observations for %s' % condition_key)
plt.xticks([0, 1], ['False', 'True'])
plt.ylabel('Observation counts');

**Self-study:** We had to convert the lists `TR_condition` and `observation_TRs_raw` into numpy arrays to make the operation above work. Why?  
 **Self-study:** Explore what happens to the observation counts when you change the `TR_buffer` and `TR_separation` variables.

In the next section we are going to take TRs that correspond to the events we have highlighted. But because of the hemodynamic lag, if we just use the timepoints we have identified, our analysis won't work. Think about an event that happens 10s into the movie: the brain won't show evidence of responding to that event for at least a few seconds because of the hemodynamic lag. 

To address this we *shift* the labels by 6s. So if an event happened at 10s, then we are going to take the brain activity at 16s to reflect the response to that event. After we have shifted the TRs, they are ready for showtime.

In [ ]:
# Cycle through the different parts of the observation_TRs_raw dictionary
TR_lag_shift = int(lag_shift / TR_duration) # Convert the lag shift into TRs
condition_key = 'is_speaking'

# Preset
observation_TRs_shifted[condition_key] = {}
labels[condition_key] = {} # What are the labels for the observations stored

for segment_name in observation_TRs_raw[condition_key].keys():

    # Add the shift to the raw data
    observation_TRs_shifted[condition_key][segment_name] = np.asarray(observation_TRs_raw[condition_key][segment_name]) + TR_lag_shift
    labels[condition_key][segment_name] = np.asarray(TR_condition[condition_key])[np.asarray(observation_TRs_raw[condition_key][segment_name])]
    

**Exercise 5:<a id="ex5"></a>** Do the same analysis so that you have `observation_TRs_shifted` and `labels` for the `'Music_Presence'` variable. In the process, create a histogram to show the counts of music on vs off (like above). Things to think about:
>Don't overwrite the existing `observation_TRs_shifted` and `labels` variables, just add your new one as a new dictionary entry  
>Make sure to perform time shifting  

In [ ]:
# Insert code here

## 3. Loading and masking our functional data <a id="mask"></a>

We have our labels, but now we need our functional data. We are going to load it in and mask it like we did back in Week 2. The mask we are going to use is the primary auditory cortex (AKA Heschls gyrus) on both the left and right hemisphere. We are using the Harvard-Oxford atlas to get this ROI.

The function below is very similar to the steps we did in week 2; however, we have additional code to pull out specific time-points, instantiated in the `select_timepoints` function. Pay close attention to this function!

In [ ]:
def prepare_sherlock(ppt, ROI_counter):
    # Load a Sherlock participant's data.
    # ppt is the participant ID e.g., sub-01
    # ROI_counter is the index of the Harvard-Oxford atlas you will use. To see all the labels, use `datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm', symmetric_split=False).labels`
    # These functions are pulled from week 2, with a tweak for loading a different mask

    # What is the file name? We are getting participant 1
    file = sherlock_dir + '/derivatives/movie_files/%s.nii.gz' % (ppt)

    # Create the nifti object that serves as a header
    func_nii = nib.load(file)

    # Get the dimensionality of the data
    func_dim = func_nii.shape

    # Get the functional voxel size
    func_voxel = func_nii.header.get_zooms()

    # Load the data volume
    print('Loading fMRI data for %s, this will take a minute' % ppt)
    func_vol = func_nii.get_fdata()
    print('Finished')

    # Specify the mask file
    mask_file = '%s/visfAtlas/nifti_volume/visfAtlas_MNI152_volume.nii.gz' % (atlas_path) 

    # Load the atlas
    atlas = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm', symmetric_split=False)

    # Get the nifti corresponding to the atlas
    mask_nii = atlas.maps

    # Run the command to align the data. Also see how Python lets you put line breaks within arguments to make it more readible!
    mask_aligned_nii = processing.conform(mask_nii, 
                                          out_shape=[func_dim[0], func_dim[1], func_dim[2]], 
                                          voxel_size=(func_voxel[0], func_voxel[0], func_voxel[0]),
                                         )

    # Pull the volume data
    mask_aligned_vol = mask_aligned_nii.get_fdata()

    # Threshold the mask
    mask_aligned_vol = mask_aligned_vol == ROI_counter

    # Produce the voxel by time matrix of the data
    func_masked = func_vol[mask_aligned_vol == 1]
    
    # Return the masked functional data
    return func_masked
    
def select_timepoints(func_masked, TR_indexes):
    # Only use the time points that we have selected
    # func_masked is the voxel by time matrix of masked data
    # TR_indexes is a dictionary containing the time points we are using for each observation
    # This returns an observation by voxel matrix
    
    func_mat = {}
    for segment_name in TR_indexes.keys():

        # What time points are you pulling out for this chunk of data
        idxs = TR_indexes[segment_name]

        # Get the voxel by time matrix for the selected time points then transpose it so that it is observation x voxel
        mat_data = func_masked[:, idxs].T

        # Store the data
        func_mat[segment_name] = mat_data

        print('Output shape for %s:' % segment_name, func_masked[:, idxs].shape)
    
    # Return the functional data
    return func_mat


Now run the functions

In [ ]:
ppt = 'sub-01' # Specify the participant to load

ROI_counter = 45 # Corresponds to Left H1 (atlas.labels.index("Heschl's Gyrus (includes H1 and H2)"))

condition_key = 'is_speaking' # Pick the condition

# Generate the voxel by time masked data
func_masked = prepare_sherlock(ppt, ROI_counter)

# Generate the observation by feature matrix you will use for analyses
func_mat[condition_key] = select_timepoints(func_masked, observation_TRs_shifted[condition_key])

In the first three sections of the notebook we have focused on setting up the data for our machine learning analysis. It is ready, so let's review the steps that were taken.  
First, we loaded in the event file which stated what happened at each TR.  
We then stored the label assigned to each individual TR.  
We then subsampled the TRs to reduce the non-independence of samples.  
We time shifted those time points so that they would index the relevant neural response for the associated time point.  
We loaded in the functional data and masked it according to a region of interest.  
Finally, we indexed the relevant time points that we have labels for, creating a voxel by time matrix that is divided based on the segment of the movie.  

With this in hand, we are now ready to turn to classification.

## 4. Classification<a id="classification"></a>

We will now build a classifier to categorize our data. 

In the example of email, spam mail has certain common features: sent from an unknown sender; sells products; asks for money deposits. These features are converted into mathematical vectors in a language feature space and computer programs are trained on these feature vectors,  _from known examples of relevant and spam emails_, to categorize emails as either relevant or spam. 

For brain activity measured by fMRI, the voxel signals serve as the features that make up the brain's response to a stimulus. We have already collected the label, namely the speaking vs not speaking label - just as the "label" of spam/not spam for emails. The voxel signals at each timepoint are features corresponding to the brain's response to the different labels. Since we know both the voxel signals and the stimulus that was presented at a given timepoint, we can train a classifier to predict when someone was speaking.

Once the classifier training is accomplished, we do not know how this classifier will perform when we give it a new set of data that the classifier didn't see during training. To determine if the classifier can _generalize_ to accurately predict data not presented during training, we test it on an _untrained_ dataset and determine its prediction accuracy. If the classifier reliably chooses the right stimulus label above chance (i.e., random guessing), we say that decoding was successful.

There are two key considerations about classification that this explanation raises that we haven't yet explored. 

First, it really matters how we decide what our training data is and what our test data is. Whereas it is "okay" for our observations to be non-independent when training our dataset (indeed, it typically hurts our performance), it is *really really important* for our training data to be independent from our test data. If you do not separate the training and test dataset, you will get inflated accuracies -- it is like taking a quiz after you have already been shown the answers. We will talk more about this in a later section. For now, know that a great way to ensure fMRI data is sufficiently independent is to **use different runs of data collection for training and test**. That is what we will do with the two segments of Sherlock data.

Second, we need to pay close attention to the counts of our two conditions. Machine learning algorithms are sneaky and may find solutions to problems in ways that we don't want. For instance, if you give an algorithm 90 observations with one label and 10 observations with another label and train it, the classifier that is learned will likely just learn to always guess the first label, since it will be right 9 times out of 10. There are ways to control for this but they are out of the scope of this notebook. Instead what we will do is make sure there are equal numbers of observations for each label.

Below we are going to split our data into training and test and then balance the labels

In [ ]:
# First we are going to rename our data to clearly demarcate what is training and what is test data and labels
condition_key = 'is_speaking'

# Get training 
train_data = func_mat[condition_key]['first_segment']
train_labels = labels[condition_key]['first_segment']

# Get test
test_data = func_mat[condition_key]['second_segment']
test_labels = labels[condition_key]['second_segment']


<div class="alert alert-block alert-info">We are using the first segment as training and second segment as test, but it would be typical to also do the reverse and then average the result. 
    </div>

In [ ]:
# Second we are going to subsample our data so that there are equal numbers in both conditions
def subsample_balance(data, labels):
    # This function makes sure that there are equal numbers of labels for the two conditions.
    # TO do this it subsamples the number of labels for the class that has more
    
    # Binarize the labels for simplicity
    labels = labels > 0
    
    # Find which label is most common
    counts = np.bincount(labels)
    most_common = np.argmax(counts)

    # How many trials of the less common label are there?
    min_count = np.min(counts)

    # Get the indexes corresponding to the most common label
    most_common_idxs = np.where(labels == most_common)[0]
    less_common_idxs = np.where(labels != most_common)[0] # Get this so you have it for later

    # Now shuffle the most common indexes and take the top N, where N is the same as the less common number
    np.random.shuffle(most_common_idxs) 
    most_common_idxs = most_common_idxs[:min_count]

    # Now concatenate the usable indexes
    usable_idxs = np.concatenate((most_common_idxs, less_common_idxs))

    # Now crop the data accordingly
    data = data[usable_idxs, :]
    labels = labels[usable_idxs, ]
    
    # Return the newly cropped data
    return data, labels

**Self-study:** In the above function we use `np.random.shuffle`. We don't give an output which is atypical for numpy. This is a "change in place" function in Python which is important to look out for

In [ ]:
# Subsample the data so that the number of each label is equal
np.random.seed(0) # Set the seed for the subsampling so that behavior is reliable for teaching purposes, be careful about seeds in general
train_data, train_labels = subsample_balance(train_data, train_labels)
test_data, test_labels = subsample_balance(test_data, test_labels)


**Exercise 6:<a id="ex6"></a>** Check that the subsampling worked. Report the counts for the two labels for training and test data, and also check the shape of the data is consistent with the shape of the labels

In [ ]:
# Insert code here

Now we can input our data into a machine learning classifier called a [support vector machine](https://en.wikipedia.org/wiki/Support-vector_machine). As you will see, after all of this prep, the actual classifier is very easy to run. 

<div class="alert alert-block alert-info">
There are many types of classifiers and each classifier has a number of parameters that can be changed to affect the sensitivity of the classification. This space is large and we are only dipping a toe in here. You can also explore a variety of classifiers <a href="http://scikit-learn.org/stable/supervised_learning.html">here</a> 
</div>

In [ ]:
# Create a linear SVC model with hyperparameter C set to 0.01.  
model = SVC(kernel="linear", C=0.01)

# Fit the model on the BOLD data and corresponding stimulus labels.
model.fit(X=train_data, y=train_labels)

# Score your model on the test data.
score = model.score(X=test_data, y=test_labels)

print('Accuracy = %0.3f' % score)

That is it! In just a couple lines we specified a classifier (specifically a linear SVM with a cost value = 0.01), we trained a classifier (on our data) and we applied it to new data in our test dataset. The classification accuracy was ~0.6 (if you didn't get this, you might have edited the data/labels for the `is_speaker` condition during Exercise 3, go back and fix it!), meaning that about 60% of the time points at test were correctly classified. We have equal numbers of labels from each condition (we ensured that with the `subsample_balance` function) and so chance is 50%. Thus we could use brain activity in auditory cortex to predict whether or not someone was speaking, better than chance!  

**Self-study:** One hyper parameter that is very important in SVM is the cost parameter (AKA the soft-margin). This governs how much the model is penalized for making errors during training. Google around to find out what the cost parameter does but [here](https://www.analyticsvidhya.com/blog/2021/10/support-vector-machinessvm-a-complete-guide-for-beginners/) is a starting point.

The `model.score` function is convenient but it doesn't give us much detail about what the classifier was doing. Below, we make a "confusion matrix" which tells us what label the model predicted and what the label should have been.

In [ ]:
# What labels does this model predict?
test_predictions = model.predict(X=test_data)

# Create a confusion matrix comparing the real and predicted labels
confusion_matrix = metrics.confusion_matrix(test_labels, test_predictions)

_, ax = plt.subplots()

ax.imshow(confusion_matrix)
plt.yticks([0, 1], ['Not speaking', 'Speaking'])
plt.xticks([0, 1], ['Not speaking', 'Speaking'])
plt.ylabel('True label')
plt.xlabel('Predicted label');

# Annotate the plot
for (j,i),label in np.ndenumerate(confusion_matrix):
        text = ax.text(j, i, confusion_matrix[i, j], ha="center", va="center", color="m", size=24)
plt.title('Confusion matrix for %s' % condition_key);

**Exercise 7<a id="ex7"></a>:** Based on the confusion matrix, comment on whether the model is biased to pick one label over the other.  
**A:**

## 5. Double dipping <a id="dipping"></a>
We said earlier that it is really important that your training and test datasets are independent. When they're non-independent, this is called **double dipping** and is a big problem. Double dipping almost always increases your classifier performance and can completely fabricate differences between conditions where none exist. Below we will consider two types of double dipping that are easy to fall prey to. The first is when the same data is used for training your model as is used for testing it. The second is when your test data is used to help pick features (in this case, voxels) that are different between conditions. In both cases, double dipping will make it seem like there are stronger differences between conditions than there are.

### Problem 1: test data used during training

If you imagine training the classifier on all of the data and then testing on a subset of data, that is double dipping. Below we do that 

In [ ]:
# Get all of your data in one place, both training and test
stacked_data = np.vstack((train_data, test_data))
stacked_labels = np.concatenate((train_labels, test_labels))

# Create a linear SVC model with hyperparameter C set to 0.01.  
junk_model = SVC(kernel="linear", C=0.01)
junk_model.fit(X=stacked_data, y=stacked_labels)
score = junk_model.score(X=test_data, y=test_labels)

print('Accuracy when testing a model on some data it was trained on: %0.3f' % score)

You can see the accuracy is really high! This is typical because machine learning algorithms are very good at finding a hyperplane that separates the data it was trained on. If you tried to use this accuracy as evidence that there is a difference between conditions then this is **double dipping and is unacceptable**. 
    
To show you just how good machine learning algorithms are at finding a hyperplane that divides the data, we are going to give an SVM junk data (i.e., labels that are unrelated to the functional data) and test the cross-validation accuracy.

In [ ]:
# Replace your training labels with junk (meaning that they are not related to the real data in any way)
junk_labels = np.copy(train_labels)
np.random.seed(0)
np.random.shuffle(junk_labels)

# Get all of your data in one place, both training and test
stacked_data = np.vstack((train_data, test_data))
stacked_labels = np.concatenate((junk_labels, test_labels))

# Make the model 
junk_model = SVC(kernel="linear", C=0.01)
junk_model.fit(X=stacked_data, y=stacked_labels)
score = junk_model.score(X=test_data, y=test_labels)

print('Accuracy when testing a model on some data it was trained on (even when the training labels are junk): %0.3f' % score)


The accuracy was still above chance! The model was able to find a hyperplane that separated observations even though there was (presumably) no coherent way in which the observations were actually clustered. This is the power of machine learning, but is also a cautionary tale. 

Now let's redo the test with junk labels but a proper training and test set division. This tests the generalization of our model and doesn't do double dipping. Since the labels are junk, this should fail

In [ ]:
# Make the model 
junk_model = SVC(kernel="linear", C=0.01)
junk_model.fit(X=train_data, y=junk_labels)
score = junk_model.score(X=test_data, y=test_labels)

print('Generalization accuracy with junk model = %0.3f' % score)

As expected, the accuracy is near chance when evaluating this junk model on held out data. In other words, the model couldn't generalize to data it hadn't been trained on. A common term to refer to a model like this is that it is "overfit" to its training data: the model is great at classifying the data it was trained on but useless on any other data.

Working through this example hopefully made it clear why double dipping is so problematic: if your training data contaminates your test data (e.g., using all the data for training a model) then your model will do well at test *even when your model is junk*.

I want to pause and reflect on the process of making `junk_labels`. By shuffling the labels and retraining the model, we are performing a critical test in machine learning. This model ought not to be able to succeed and so if it ever does, like in the example above, then that is cause for concern. When you are developing your analyses, you should do tests that shuffle your labels to check your results are near chance.

### Problem 2: test data used to help preprocess the training data

The other kind of double dipping that is important to recognize is when an operation, typically feature selection, is applied to *all* of your data *before* splitting it into training and test datasets. 

Imagine you show participants images of faces and scenes (what we call a functional localizer). Different regions of the brain typically respond to these stimulus categories, but the location of these regions varies slightly across participants. In your analysis you might want to first identify the voxels that respond to each stimulus category using traditional univariate analyses on the whole dataset (kind of like a t test between the responses to the stimulus categories). This process is called voxel selection, feature selection or functional ROI definition. 

If your voxel selection was done on *all* of your data and then your data was split into training and test, **this is double-dipping and you should never do it**, even if you follow other suggestions like using different runs of data for training and test. The reason is that the voxel selection procedure already confirmed that there is a difference between conditions in those voxels, so running a decoding analysis to test that there is a difference in those voxels is circular and sure to succeed. In other words, if there is no effect, then this process will fabricate one. 

Below we will do a demo of this kind of double-dipping with some simulated data with no difference between conditions.

In [ ]:
# Simulate some data that is observations (think of trials of faces and scenes) by voxel.
# This data is purely gaussian noise
np.random.seed(0)
train_sim = np.random.randn(40, 20)
test_sim = np.random.randn(40, 20)

# Now lets arbitrarily divide our data into conditions (e.g., faces vs scenes)
labels_sim = np.asarray([0] * 20 + [1] * 20)

# Run the classifier on this data
model = SVC(kernel="linear", C=0.01)
model.fit(X=train_sim, y=labels_sim)
score = model.score(X=test_sim, y=labels_sim)

print('Accuracy for decoding simulated data = %0.3f' % score)

So with simulated data that has no signal in it (i.e., no difference between conditions), our accuracy is near chance, as expected. Now we are going to do voxel selection on our data and then rerun this decoding. 

In [ ]:
# Combine all of our data (this is exactly what you SHOULD NOT DO for feature selection)
stacked_data = np.vstack((train_sim, test_sim))
stacked_labels = np.concatenate((labels_sim, labels_sim))

# Run a t test comparing each individual voxel between the two conditions
condition_diffs = stats.ttest_ind(stacked_data[stacked_labels == 1, :], stacked_data[stacked_labels == 0, :]).statistic

# Rank the voxels in their difference between the conditions from smallest to biggest (we use the absolute difference because both signs are informative)
ranked_diffs = np.argsort(np.abs(condition_diffs))

# Take the top 3 voxels
best_voxels = ranked_diffs[-3:]

# Run the classifier
model = SVC(kernel="linear", C=0.01)
model.fit(X=train_sim[:, best_voxels], y=labels_sim)
score = model.score(X=test_sim[:, best_voxels], y=labels_sim)

print('Accuracy for decoding simulated data = %0.3f' % score)

Now our accuracy is substantially above chance when working with noise! This is because we found voxels that have a difference between conditions and then tested whether they have a difference between conditions. This is textbook double-dipping.

**Exercise 8<a id="ex8"></a>:** Do feature selection correctly on the simulated data. Specifically, only run feature selection on the training data. Interpret the decoding accuracy you get.

In [ ]:
# Insert code here

So it's clear we need to be cautious with feature selection but I don't want to completely discourage feature selection. Feature selection has value when you have very high dimensional data (i.e., a lot of voxels) that are noisy and is a necessary way to overcome the curse of dimensionality. To learn more about feature selection, consider doing this [brainiak tutorial](https://brainiak.org/tutorials/04-dimensionality-reduction/) on it. Just make sure you always run your feature selection on only your training data, protecting your test data. 

Feature selection is a common example of a more general case of double dipping in which *any* operation is applied to all your data. In preprocessing fMRI data we often apply filtering or demeaning operations to our entire run. That means that *technically* using training and test data from the same run is violating independence and is double-dipping. This is one of the reasons you should always use separate runs of data for training and test. That said, these kinds of preprocessing operations (as well as things like z scoring) are usually harmless... usually... 

<div class="alert alert-block alert-info">
The above example about preprocessing raises an important point: <strong>always check your data by shuffling labels</strong>. Shuffling your labels should eliminate the differences between conditions, so if you can still decode accurately, it is likely because of your training data is leaking into your test.
    
</div>

## 6. Classification in all participants<a id="between"></a>

So above we could classify between speaking and non-speaking at about 60% accuracy. That is greater than 50%, but it isn't a particularly high number, so how do we know if 60% is meaningful? 

There are lots of options to evaluate the meaningfulness of a decoding accuracy. One thing you could do is what we did above: shuffle the labels so that they don't relate to your data and see how typical it is to get a decoding accuracy is 60%. This is called permutation testing and is a strong approach.

An alternative option is to run these analyses for each individual participant. If ALL your participants show classification accuracy over 50%, that is pretty compelling evidence that your decoding was successful.

<div class="alert alert-block alert-info">
Note: consistent performance across participants doesn't tell us anything about double dipping: all of your data could be double-dipped and thus above chance!
    </div>

**Exercise 9<a id="ex9"></a>:** Put the relevant code together from above to find the classification accuracy for the 17 participants in the Sherlock dataset. Compile the classification accuracies and report the mean, minimum and maximum. Test whether the accuracy is significantly different from chance using a t-test. Finally, interpret the results you get: what does this mean?

In [ ]:
# Insert code here

**Self-study:** As mentioned, we are just scratching the surface of methods that we can use to perform classification. There are many more tools for classification. I will briefly introduce a few below, but if you want a tutorial, then [this notebook](https://brainiak.org/tutorials/05-classifier-optimization/) is a good place to start

>`GridSearchCV`: Search over a grid (i.e., a range) of classification parameters and find the optimal features. Make sure to use a nested cross-validation sample! Read section 3.3 of the notebook linked above  
>sklearn splits: Read [here](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection) for dozens of options to split your data into training and test according various criteria.   
>`Pipeline`: Create a pipeline for preprocessing and classifying your data. This is a highly flexible tool that can automate your workflow.  
>Regularization: L1 and L2 regularization are different ways of penalizing a model for its complexity

**Novel contribution:** <a id="novel"></a> be creative and make one new discovery by adding an analysis, visualization, or optimization. 


In [ ]:
# Put novel contribution here

## Contributions <a id="contributions"></a> 
 
C. Ellis Adapted the Brainiak tutorial on classification to include movie datasets, blending content from multiple notebooks